In [63]:
!pip install kafka-python
!pip install avro

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [64]:
# import required libraries
from kafka import KafkaConsumer, KafkaProducer
import avro.schema
import avro.io
import io
import hashlib, json

In [65]:
def serialize(schema, obj):
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    writer = avro.io.DatumWriter(schema)
    writer.write(obj, encoder)
    return bytes_writer.getvalue()

In [66]:
def deserialize(schema, raw_bytes):
    bytes_reader = io.BytesIO(raw_bytes)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    reader = avro.io.DatumReader(schema)
    return reader.read(decoder)

In [67]:
schema_file = 'transaction.avsc'
txschema = avro.schema.parse(open(schema_file).read())
schema_file = 'submit.avsc'
submitschema = avro.schema.parse(open(schema_file).read())
schema_file = 'result.avsc'
resultschema = avro.schema.parse(open(schema_file).read())

In [68]:
# Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
kafka_broker = '35.240.149.229:9092'

In [69]:
producer = KafkaProducer(bootstrap_servers=[kafka_broker])

In [70]:
txconsumer = KafkaConsumer(
    'transaction',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(txschema, x))
resultconsumer = KafkaConsumer(
    'result',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(resultschema, x))

In [71]:
def gen_signature(txid, payer, payee, amount, token):
    o = {'txid': txid, 'payer': payer, 'payee': payee, 'amount': amount, 'token': token}
    return hashlib.md5(json.dumps(o, sort_keys=True).encode('utf-8')).hexdigest()

# Read a transaction from “transaction” topic using transaction.avsc schema, extract transaction information including txid, payer, payee, and amount

In [72]:
message = next(txconsumer)

txid = message.value['txid']
payer = message.value['payer']
payee = message.value['payee']
amount = message.value['amount']

print('Transaction ID = ' ,txid, "\n")
print('Payer = ', payer, "\n") 
print('Payee = ' ,payee, '\n')
print('Amount = ' ,amount)

Transaction ID =  TX07492 

Payer =  A01809 

Payee =  A57652 

Amount =  707


# Verify transaction information with your token by creating signature using gen_signature function (given in the assignment) and submit your verification (vid, txid, and signature) to “submit” topic using submit.avsc schema

In [73]:
token = 'effe89619df4c3790d24a4b92a03ade3'
vid = 'V136728' 
signature = gen_signature(txid, payer, payee, amount, token)
submit = {"vid": vid, "txid": txid, "signature": signature}
raw_bytes = serialize(submitschema, submit)
producer.send('submit', value=raw_bytes)


# Verifier receives a confirmation from “result” topic (message with code “200” with matching vid and txid

In [74]:
for message in resultconsumer:

    if message.value["vid"] == vid and message.value["txid"] == txid and message.value["code"] == 200:
        print(message.value)
        break
        
resultconsumer.close()

{'timestamp': 1679934155, 'vid': 'V136728', 'txid': 'TX07492', 'checksum': 'c867427b2ead92083b5319359607c6e6', 'code': 200, 'message': 'Confirm'}
